# Problem 96: Su Doku
https://projecteuler.net/problem=96

## Solution
We can solve this problem as an integer linear program (ILP). Define $$x_{ijk} =\begin{cases}1 \text{ if number $k$ on row $i$ and column $j$} \\ 0 \text{ otherwise}\\\end{cases},$$ then we can add constraints for each row, each column and each 3x3 block. We also fill in the filled out squares as constraints. Note that there is no objective function.

# Importing data

In [98]:
f = [x.rstrip() for x in open("pe096.txt").readlines()]
sudokus = [f[(i*10)+1:(i+1)*10] for i in range(len(f)//10)]

# Solver algorithm

In [95]:
from pulp import *
import numpy as np
from itertools import product

def solve(sudoku):
    """Solves an incomplete sudoku and retrieves the concatenation three top left numbers."""
    model = LpProblem("sudoku", LpMinimize)
    
    # Decision variables
    x = LpVariable.dicts("x", (range(1,10),range(1,10),range(1,10)),cat="Binary")
    
    # Objective function
    model += 0
    
    # Each column must contain number k exactly once
    for j in range(1,10):
        for k in range(1,10):
            model += lpSum([x[i][j][k] for i in range(1,10)]) == 1
            
    # Each row must contain number k exactly once
    for i in range(1,10):
        for k in range(1,10):
            model += lpSum([x[i][j][k] for j in range(1,10)]) == 1
            
    # Each element must contain at least one number
    for i in range(1,10):
        for j in range(1,10):
            model += lpSum([x[i][j][k] for k in range(1,10)]) == 1
    
    # Each 3x3 block must contain number k exactly once
    blocks = list(product([(1,4), (4,7), (7,10)],repeat=2))
    for b in blocks:
        for k in range(1, 10):
            model += lpSum([x[i][j][k] 
                            for i in range(b[0][0], b[0][1]) 
                            for j in range(b[1][0], b[1][1])]) == 1
    
    # Add the constraints from the data
    for i in range(9):
        for j in range(9):
            k = int(sudoku[i][j])
            if k != 0:
                model += x[i+1][j+1][k] == 1
    
    # Solve
    model.solve(solver=solvers.GUROBI(msg=0))
    
    # Store the top left corner numbers
    tlc = ''  
    for j in range(1,4):
        for k in range(1,10):
            if x[1][j][k].varValue == 1:
                tlc += str(k)
    return(int(tlc))

# Final algorithm

In [100]:
def pe096(sudokus):
    tlc = [] # Top left corner numbers    
    for sudoku in sudokus:
        tlc.append(solve(sudoku))
    return(sum(tlc))

In [99]:
pe096(sudokus)

24702